In [1]:
import angr
from angrutils import *

In [555]:
# binary = '/data2/zhangzheng1/data/example-c/build/src/test_string'
# binary = './test'
binary = '/usr/bin/base64'

p = angr.Project(binary, auto_load_libs=False)

In [545]:
# main = p.loader.main_object.get_symbol("main")
# print(hex(main.rebased_addr))
# print(hex(p.entry))

In [556]:
# cfg = p.analyses.CFGEmulated(fail_fast=True, starts=[main.rebased_addr])
cfg = p.analyses.CFGFast()

WARNING | 2020-07-02 12:14:44,430 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00a0 with 8 unconstrained bytes referenced from 0x4031b1 (PLT.__gmon_start__+0x1a81 in base64 (0x4031b1))
WARNING | 2020-07-02 12:14:44,433 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00a8 with 8 unconstrained bytes referenced from 0x4031b2 (PLT.__gmon_start__+0x1a82 in base64 (0x4031b2))
WARNING | 2020-07-02 12:14:44,436 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00b0 with 8 unconstrained bytes referenced from 0x4031b3 (PLT.__gmon_start__+0x1a83 in base64 (0x4031b3))
WARNING | 2020-07-02 12:14:44,438 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00b8 with 8 unconstrained bytes referenced from 0x4031b5 (PLT.__gmon_start__+0x1a85 in base64 (0x4031b5))
WARNING | 2020-07-02 12:14:44,440 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00c0 with 8 unconstrained bytes referenced from 0x4031b7

In [557]:
# cfg.kb.functions.get('main').get_call_sites()

In [558]:
# for b in sorted(cfg.kb.functions.get('main').blocks, key=lambda x: x.addr):
#     b.pp()
#     print()

In [559]:
# [b for b in cfg.kb.functions.get('__libc_start_main').blocks][0].pp()

In [560]:
# cfg.kb.functions.get('main')

In [561]:
def checkNop(blk): # 检查blk全部为nop的情况
    try:
        if not blk: return True
        if blk.size == 0: return True
        return all([blk.capstone.insns[i].insn.insn_name() == 'nop' for i in range(blk.instructions)])
    except Exception as e:
        print('[NOP]{} {}'.format(blk, blk.size))
        return False

def getcov(func, s):

    blkList = list(func.blocks)

    for blk in blkList: # 遍历该函数所有block
        if blk.addr not in s:
            s.add(blk.addr)
            
            node = cfg.get_node(blk.addr) # 将block转换为node

            if node.has_return or checkNop(blk): # 该block包含ret则不遍历successors或全部为nop指令
                s.add(node.addr)
            else:
                for ns in node.successors: # 遍历该block的后续block
                    if ns.addr not in s:
                        if ns.has_return or checkNop(ns.block): # 同上
                            s.add(ns.addr)
                        else:
                            if ns.addr in func.block_addrs: # func内条件跳转（angr解析func得到的blk会少一部份）
                                continue
                            else:
                                try:
                                    new_func = cfg.kb.functions.get_by_addr(ns.addr) # jmp或call调用其它函数
                                    getcov(new_func, s) # 递归解析下一个函数
                                except KeyError as e: # func内跳转到angr未解析为该func的blk
                                    blkList.append(ns.block)
#                                     print('[!] {} {} {}'.format(func.name, hex(blk.addr), hex(ns.addr)))
                                    continue
    
    return len(s)

In [562]:
print('[+] Functions: {}'.format(len(list(cfg.kb.functions.items()))))
print('[+] Nodes: {}'.format(len(cfg.graph.nodes())))

[+] Functions: 381
[+] Nodes: 1495


In [553]:
from tqdm import tqdm

func2cov = {}

for addr, func in tqdm(cfg.kb.functions.items()):
    if func.name not in func2cov:
        func2cov[func.name] = getcov(func, set())
    else:
        func2cov[func.name] = max(func2cov[func.name], getcov(func, set()))

100%|██████████| 381/381 [00:02<00:00, 133.23it/s]


In [563]:
main_cov = getcov(cfg.kb.functions.get('main'), set())
all_blk = len(cfg.graph.nodes())
print('[+] {:.2%}'.format(main_cov/all_blk))

[+] 58.73%


In [526]:
# sub_403162 0x40318f 0x403b9f
# func = cfg.functions.get('sub_402e10')
# # print(0x402e10 <= 0x403a10 < 0x40343f)
# for b in sorted(func.blocks, key=lambda x: x.addr):
# # for b in func.blocks:
# #     if b.addr == 0x40336a:
#     b.pp()
#     print()
# print(hex(func.addr))
# print(hex(func.addr+func.size))
# print(func.size)

In [527]:
# 0x403162 in func.block_addrs

In [528]:
# cfg.get_node(0x403162).block.pp()

In [529]:
main = cfg.functions.get('main')
blkcnt = 0
for blk in main.blocks:
    blkcnt += 1
    for s in cfg.get_node(blk.addr).successors:
        if not (main.addr <= s.addr <= main.addr + main.size):
            name = cfg.functions.get_by_addr(s.addr).name
            print('{} {}'.format(name, func2cov[name]))
print('main: {} {}'.format(blkcnt, func2cov['main']))

sub_402c10 20
setlocale 2
bindtextdomain 2
textdomain 2
sub_405a30 5
getopt_long 2
sub_4044f0 331
dcgettext 2
fopen 2
error 2
sub_4049b0 63
sub_4020a0 63
exit 2
sub_4020a0 63
dcgettext 2
sub_402be0 8
sub_4042d0 342
sub_404e30 403
__errno_location 2
sub_401d70 98
sub_401ae0 135
error 2
sub_405400 46
sub_4042d0 342
dcgettext 2
__errno_location 2
__errno_location 2
error 2
error 2
main: 55 878


In [530]:
for f in sorted(func2cov, key=lambda x: func2cov[x], reverse=True)[:40]:
    print('{}: {}'.format(f, func2cov[f]))

main: 878
sub_40366a: 468
sub_402b20: 409
sub_404e30: 403
sub_404d20: 401
sub_4040e2: 344
sub_404143: 344
sub_4041b0: 344
sub_4041c0: 344
sub_4040f7: 342
sub_40415c: 342
sub_4042d0: 342
sub_404450: 338
sub_404460: 338
sub_404480: 338
sub_404390: 337
sub_4041cc: 335
sub_404290: 332
sub_4042a0: 332
sub_4042c0: 332
sub_404080: 331
sub_4040a0: 331
sub_4040b0: 331
sub_4040d0: 331
sub_4044a0: 331
sub_4044b0: 331
sub_4044d0: 331
sub_4044f0: 331
sub_403bb0: 330
sub_403ed2: 304
sub_403fd0: 304
sub_403efb: 302
sub_4037bb: 284
sub_403e60: 284
sub_403771: 282
sub_402e10: 279
sub_402e9a: 279
sub_403162: 279
sub_402d40: 174
sub_4055a0: 156


In [113]:
# plot_cfg(cfg, "test", asminst=True, remove_imports=True, remove_path_terminator=True)